# Step 1: configuration

Input your walletPrivateKey here:

In [1]:
# make sure you configure mindlakesdk, please check env.py in the same folder
# please familar these configure, and you can read https://github.com/mind-network/mind-lake-sdk-python 
# You can get web3StorageToken from https://web3.storage/
import env

print("check env.walletPrivateKey:", env.walletPrivateKey[0:5])
print("check env.mindLakeAppKey:", env.mindLakeAppKey[0:5])
print("check env.MINDLAKE_GATEWAY:", env.MINDLAKE_GATEWAY)
print("check env.web3StorageToken:", env.web3StorageToken[0:5])

check env.walletPrivateKey: 2a776
check env.mindLakeAppKey: Kyunq
check env.MINDLAKE_GATEWAY: https://sdk.mindnetwork.xyz/node
check env.web3StorageToken: eyJhb


Install depedancy and source code

In [2]:
# please "pip install mindlakesdk" if not installed
import mindlakesdk

# please "pip install minddatapack" if not installed, or git clone the source code
from minddatapack import DataPack

# please "pip install pandas" if not installed, it is used to display structure content in the notebook
import pandas

In [3]:
# check if example_data.csv file exists in the same folder, this demo will start to load this csv file.

## Step 2. Local CSV -> MindLake
- Load data from a local CSV file without the metadata file, by defining each column manually.
- You can also load from a local backup into MindLake once you are familar, then no need to define each column. 

In [4]:
fn_local_csv = './example_data.csv'

# preview local csv
df = pandas.read_csv(fn_local_csv)
df

,uid,wallet_address,register_date
0,123,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
1,124,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
2,125,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441


In [5]:
dataPack1 = DataPack(env.walletPrivateKey)
localFile1 = DataPack.LocalFileConnector(columns=[
        DataPack.Column('uid', DataPack.DataType.int4, False),
        DataPack.Column('wallet_address', DataPack.DataType.text, True),
        DataPack.Column('register_date', DataPack.DataType.timestamp, True)
    ])
result = dataPack1.loadFrom(localFile1, fn_local_csv)
assert result, result.message
df = pandas.DataFrame(dataPack1.data)
df

,0,1,2
0,123,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
1,124,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
2,125,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441


- Save the data into a table in MindLake

In [6]:
mindlake = mindlakesdk.connect(env.walletPrivateKey, env.mindLakeAppKey, env.MINDLAKE_GATEWAY)
assert mindlake, mindlake.message

# drop the table if exists
result = mindlake.datalake.dropTable('a_new_table')
print("dropTable result:", result.message)

# upload from local csv into mindlake
mindLakeConnector = DataPack.MindLakeConnector(mindlake)
result = dataPack1.saveTo(mindLakeConnector, 'a_new_table')
assert result, result.message
print("a_new_table has been saved to MindLake.")

dropTable result: Success
a_new_table has been saved to MindLake.


you can also check this new downloaded table in https://scan.mindnetwork.xyz/account/myData

## Step 3. MindLake -> Web3Storage
- Load data from a table in MindLake

In [7]:
dataPack2 = DataPack(env.walletPrivateKey)
# make sure if you complete Step 1, so "a_new_table" exists in mindlake
result = dataPack2.loadFrom(mindLakeConnector, 'select * from "a_new_table"')
assert result, result.message
df = pandas.DataFrame(dataPack2.data)
df

,0,1,2
0,123,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
1,124,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
2,125,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441


- Save the data into Web3Storage

In [8]:
web3Storage1 = DataPack.Web3StorageConnector(env.web3StorageToken)
result = dataPack2.saveTo(web3Storage1, 'test_table_encrypted.csv')
assert result, result.message
print("test_table_encrypted.csv has been saved to Web3Storage.\n")
ipfsCid = result.data
print("The IPFS CID is:", ipfsCid, '\n')
ipfs_file_url = "https://w3s.link/ipfs/%s"%ipfsCid
ipfs_file_info_url = "https://api.web3.storage/user/uploads/%s"%ipfsCid
print(f"You can download the archive file from: {ipfs_file_url}\n")
print("Here is storage info of the archive file:")
import requests
headers = {
    "Authorization": "Bearer " + env.web3StorageToken
}
content = requests.get(ipfs_file_info_url, headers=headers).text
print(content)

test_table_encrypted.csv has been saved to Web3Storage.

The IPFS CID is: bafkreigsgm7gmydxagcuo3w6jtnsm36kvdoiml3dtfjomy6ksioqe6d3n4 

You can download the archive file from: https://w3s.link/ipfs/bafkreigsgm7gmydxagcuo3w6jtnsm36kvdoiml3dtfjomy6ksioqe6d3n4

Here is storage info of the archive file:
{"_id":"315318962443259193","type":"Upload","name":"Upload at 2023-09-03T09:09:01.164Z","created":"2023-09-03T09:09:01.164+00:00","updated":"2023-09-03T09:09:01.164+00:00","cid":"bafkreigsgm7gmydxagcuo3w6jtnsm36kvdoiml3dtfjomy6ksioqe6d3n4","dagSize":1626,"pins":[{"status":"Pinned","updated":"2023-09-03T09:09:01.164+00:00","peerId":"bafzbeibhqavlasjc7dvbiopygwncnrtvjd2xmryk5laib7zyjor6kf3avm","peerName":"elastic-ipfs","region":null}],"deals":[]}


## Step 4. Web3Storage -> Local File
- Load data from Web3Storage and save to local file

In [9]:
dataPack3 = DataPack(env.walletPrivateKey)
web3Storage2 = DataPack.Web3StorageConnector()
result = dataPack3.loadFrom(web3Storage2, ipfsCid)
assert result, result.message
print("You can see file on Web3Storage are encrypted, but datapack can decrypted it locally")
df = pandas.DataFrame(dataPack3.data)
df

You can see file on Web3Storage are encrypted, but datapack can decrypted it locally


,0,1,2
0,123,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
1,124,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
2,125,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441


- Save the data into a local file with encryption

In [10]:
localFile2 = DataPack.LocalFileConnector()
result = dataPack3.saveTo(localFile2, 'test_localtable_encrypted.csv')
assert result, result.message
print("test_localtable_encrypted.csv has been saved to local file.")
df = pandas.read_csv('test_localtable_encrypted.csv')
df

test_localtable_encrypted.csv has been saved to local file.


,uid,wallet_address,register_date
0,123,\xec58be677e5c22bac7ea266df234565fe56506aa0d5e...,\x1ef6e317b5cbfe4568fc648dad1bb9e7e8e872005071...
1,124,\xa55d391505fbb845d77990d3c415dbed2d758b7d3977...,\xd460941b3d1cbb9bbd1014a02712b18a918b869af046...
2,125,\xcc8e1cffefa3029ecb20518c098254aa421d8904f92a...,\xe8d4aac12f92c75cd1589b028a7ae5f3e9c47c1259f6...
